In [1]:
def read_file(filepath):
  with open(filepath) as f:
    str_text = f.read()
    
    return str_text

In [8]:
filename = "/Users/test/Downloads/UPDATED_NLP_COURSE/06-Deep-Learning/moby_dick_four_chapters.txt"

In [ ]:
read_file()

In [2]:
import spacy

In [4]:
nlp = spacy.load("en_core_web_sm")

In [5]:
nlp.max_length = 1198623

In [22]:
def separate_punc(doc_text):
  return [token.text.lower() for token in nlp(doc_text) if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n']

In [9]:
d = read_file(filename)

In [23]:
tokens = separate_punc(d)

In [24]:
len(tokens)

11338

In [25]:
train_len = 25 +1

text_seq = []

for i in range(train_len, len(tokens)):
  seq = tokens[i-train_len:i]
  text_seq.append(seq)

In [26]:
from keras.preprocessing.text import Tokenizer

In [27]:
tokenizer = Tokenizer()

In [28]:
tokenizer.fit_on_texts(text_seq)
seq = tokenizer.texts_to_sequences(text_seq)
seq

[[956,
  14,
  263,
  51,
  261,
  408,
  87,
  219,
  129,
  111,
  954,
  260,
  50,
  43,
  38,
  314,
  7,
  23,
  546,
  3,
  150,
  259,
  6,
  2713,
  14,
  24],
 [14,
  263,
  51,
  261,
  408,
  87,
  219,
  129,
  111,
  954,
  260,
  50,
  43,
  38,
  314,
  7,
  23,
  546,
  3,
  150,
  259,
  6,
  2713,
  14,
  24,
  957],
 [263,
  51,
  261,
  408,
  87,
  219,
  129,
  111,
  954,
  260,
  50,
  43,
  38,
  314,
  7,
  23,
  546,
  3,
  150,
  259,
  6,
  2713,
  14,
  24,
  957,
  5],
 [51,
  261,
  408,
  87,
  219,
  129,
  111,
  954,
  260,
  50,
  43,
  38,
  314,
  7,
  23,
  546,
  3,
  150,
  259,
  6,
  2713,
  14,
  24,
  957,
  5,
  60],
 [261,
  408,
  87,
  219,
  129,
  111,
  954,
  260,
  50,
  43,
  38,
  314,
  7,
  23,
  546,
  3,
  150,
  259,
  6,
  2713,
  14,
  24,
  957,
  5,
  60,
  5],
 [408,
  87,
  219,
  129,
  111,
  954,
  260,
  50,
  43,
  38,
  314,
  7,
  23,
  546,
  3,
  150,
  259,
  6,
  2713,
  14,
  24,
  957,
  5,
  60,
  5,
  5

In [35]:
vocab_size = len(tokenizer.word_counts)

In [30]:
import numpy as np

seq = np.array(seq)

In [31]:
from keras.utils import to_categorical

In [61]:
X = seq[:,:-1]

In [86]:
y = seq[:,-1]

In [87]:
y = to_categorical(y, num_classes=vocab_size+1)

In [38]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding

In [88]:
def create_model(vocab_size, seq_len):
  model = Sequential()
  model.add(Embedding(vocab_size, seq_len, input_length=seq_len))
  model.add(LSTM(50, return_sequences=True))
  model.add(LSTM(50))
  model.add(Dense(50, activation='relu'))
  model.add(Dense(vocab_size, activation='softmax'))
  
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  
  model.summary()
  return model

In [89]:
model = create_model(vocab_size+1,X.shape[1])

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 25, 25)            68000     
                                                                 
 lstm_18 (LSTM)              (None, 25, 50)            15200     
                                                                 
 lstm_19 (LSTM)              (None, 50)                20200     
                                                                 
 dense_18 (Dense)            (None, 50)                2550      
                                                                 
 dense_19 (Dense)            (None, 2720)              138720    
                                                                 
Total params: 244,670
Trainable params: 244,670
Non-trainable params: 0
_________________________________________________________________


In [91]:
model.fit(X,y,batch_size=128,epochs=50, verbose=1)

Epoch 1/50
89/89 [==============================] - 3s 37ms/step - loss: 4.6200 - accuracy: 0.1236
Epoch 2/50
89/89 [==============================] - 3s 32ms/step - loss: 4.5775 - accuracy: 0.1214
Epoch 3/50
89/89 [==============================] - 3s 33ms/step - loss: 4.5327 - accuracy: 0.1263
Epoch 4/50
89/89 [==============================] - 3s 32ms/step - loss: 4.4907 - accuracy: 0.1289
Epoch 5/50
89/89 [==============================] - 4s 43ms/step - loss: 4.4516 - accuracy: 0.1310
Epoch 6/50
89/89 [==============================] - 7s 77ms/step - loss: 4.4092 - accuracy: 0.1315
Epoch 7/50
89/89 [==============================] - 4s 39ms/step - loss: 4.3581 - accuracy: 0.1365
Epoch 8/50
89/89 [==============================] - 2s 28ms/step - loss: 4.3210 - accuracy: 0.1377
Epoch 9/50
89/89 [==============================] - 2s 27ms/step - loss: 4.2866 - accuracy: 0.1386
Epoch 10/50
89/89 [==============================] - 2s 27ms/step - loss: 4.2487 - accuracy: 0.1413
Epoch 11/

In [92]:
from pickle import dump


model.save("my_model.h5")

dump(tokenizer, open("my_tokenier", 'wb'))

In [95]:
from keras.utils import pad_sequences

def generate_text(model, tokenizer, seq_len, seed_text, num_gen_words):
   ouput_text = []
   input_text = seed_text
   
   for i in range(num_gen_words):
     encoded_text = tokenizer.text_to_sequences([input_text])[0]
     
     pad_encoded = pad_sequences([encoded_text], max_len=seq_len, truncating='pre')
     
     pred_prob = model.predict(pad_encoded, verbose=0)[0]
     
     pred_word_ind = np.argmax(pred_prob)
     
     pred_word = tokenizer.index_word[pred_word_ind]
     
     input_text += " " + pred_word
     ouput_text.append(pred_word)
   
   return ' '.join(ouput_text)

In [96]:
import random
random.seed(101)

random_pick = random.randint(0, len(text_seq))


In [97]:
random_seed_text = text_seq[random_pick]

In [98]:
seed_text = " ".join(random_seed_text)